In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5
import numpy as np
import time
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import analysis.switching as sw

In [3]:
from experiments.Pulse_Switching_CSHE_BER_nTron import nTronBERExperiment

In [4]:
exp = nTronBERExperiment()
exp.sample = "CSHE5 - C1R3"
exp.comment = "nTron Bit Error Rate - AP to P - 5ns"
exp.polarity = -1 # -1: AP to P; 1: P to AP
exp.field.value = -0.0074
exp.nTron_duration.value = 5e-9 # Fixed
exp.reset_amplitude = 0.2
exp.reset_duration  = 5.0e-9
exp.settle_delay    = 200e-6
exp.init_instruments()

In [5]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die5-C1R3\CSHE5-C1R3-AP2P_2016-07-20_BER_nTron_5ns.h5")
edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)

In [6]:
attempts_list = [1 << int(x) for x in np.linspace(22,22,2)]
voltages_list = np.linspace(0.9,0.95,2)
t1 = [] # Keep track of time
t2 = []
for att, vol in tqdm_notebook(zip(attempts_list, voltages_list), total=len(voltages_list)):
    print("Now at ({},{}).".format(att,vol))
    t1.append(time.time())
    exp.attempts.value = att
    exp.nTron_voltage.value = vol
    exp.init_streams()
    exp.reset()
    exp.run_loop()
    t2.append(time.time())
    print("Elapsed time: {} min.".format((t2[-1]-t1[-1])/60))

Now at (4194304,0.9).
Elapsed time: 40.11975325743357 min.
Now at (4194304,0.95).
Elapsed time: 40.92621033986409 min.



In [7]:
print("Total elapsed time: {} min.".format((t2[-1]-t1[0])/60))

Total elapsed time: 81.0460137406985 min.


In [8]:
# Plot data
data_mean = sw.load_BER_data(wr.filename)
fig = sw.plot_BER(voltages_list, data_mean)
plt.show()

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)
